In [1]:
from xgboost import XGBClassifier
import lightgbm as lgb
#from catboost import CatBoostClassifier

import matplotlib.pyplot as plt
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import pandas as pd
import numpy as np
import joblib
from datetime import datetime
import sys

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GroupKFold
from sklearn.ensemble import RandomForestClassifier

# Add the root of the project to the path
source_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(source_path)


In [50]:
possible_inputs ={'kaggle': 'icdar_train_df_KAGGLE_20250514_181737.csv','original':'icdar_train_df_20250514_175905.csv', 
                  'patch_dataset':'icdar_train_df_patches_20250515_164130.csv',
                  'smaller_patches':'icdar_train_df_patches_20250521_120324.csv', }
output_dir = source_path + "\\outputs\\preprocessed_data\\"
LOG_FILE = output_dir+"file_metadata_log.json"


In [51]:
extracted_from= 'patch_dataset'  # 'original', 'patch_dataset', 'smaller_patches'
if extracted_from not in possible_inputs:
    raise ValueError(f"Invalid input type: {extracted_from}. Choose from {list(possible_inputs.keys())}.")
else:
    source_data = possible_inputs[extracted_from]
# i select the model and get all the instances in which I used that model on that data
# for each instance i get the transforms related data and return the info on the transforms  
file_IO.show_model_instances(LOG_FILE,['model',"transform_mode","custom transform","description"], source_data )

Metadata for icdar_EXTRACTED_train_df_trocr-small-stage1_20250516_122528.csv:
model: trocr-small-stage1
description: I am performing inference with trocr-small-stage1 on the patches dataset and saving the output features as columns of the dataframe
        the columns are named f1, f2, .., f384
------------------------------------------
Metadata for icdar_EXTRACTED_train_df_trocr-small-handwritten_20250516_150814.csv:
model: trocr-small-handwritten
description: I am performing inference with trocr-small-handwritten on the patches dataset and saving the output features as columns of the dataframe
        the columns are named f1, f2, .., f384
------------------------------------------
Metadata for icdar_EXTRACTED_train_df_resnet50_20250516_161022.csv:
model: resnet50
custom transform: False
description: I am performing inference with resnet50 on the patches dataset and saving the output features as columns of the dataframe
        the columns are named f1, f2, .., f2048. I apply the sta

In [52]:
# I select the transform and load the input file corresponding to that model and transform (the most recent one)
model_used = 'clip-vit-large-patch14'
transform_used = ''
file_IO.get_file_name(LOG_FILE,{'model':model_used},source_data)

icdar_EXTRACTED_train_df_clip-vit-large-patch14_20250517_144404.csv


In [33]:
input_file_name='icdar_EXTRACTED_train_df_clip-vit-large-patch14_20250604_201425.csv'
input_file=source_path+'\\outputs\\preprocessed_data\\'+input_file_name
train_FE = pd.read_csv(input_file)

In [34]:
train_FE = file_IO.change_filename_from_to(train_FE, fr="old-laptop", to="new-laptop")

In [35]:
def select_n_patches(train_df, n_patches=10):
    grouped_sorted = train_df.groupby('page', group_keys=False).apply(lambda x: x.sort_values('black_ratio', ascending=False))
    grouped_sorted = grouped_sorted.groupby('page', group_keys=False).head(n_patches)
    return grouped_sorted

In [36]:
train_FE['page'] = train_FE.groupby(['writer', 'isEng', 'same_text']).ngroup()

In [37]:
n_patches = 1
if n_patches > 0:
    #print(f"Selecting {n_patches} patches per page...")
    train_FE = select_n_patches(train_FE, n_patches=n_patches).reset_index(drop=True)

C:\Users\andre\AppData\Local\Temp\ipykernel_34524\1653312610.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_sorted = train_df.groupby('page', group_keys=False).apply(lambda x: x.sort_values('black_ratio', ascending=False))


In [38]:
train_FE

,writer,isEng,same_text,file_name,male,train,index,x,y,x2,...,f760,f761,f762,f763,f764,f765,f766,f767,f768,page
0,1,0,0,C:\Users\andre\PhD\Datasets\ICDAR 2013 - Gende...,0,1,0,1722,984,1968,...,-0.063661,-0.004135,0.008908,-0.031945,0.025556,-0.018130,0.020174,-0.004420,-0.005928,0
1,1,0,1,C:\Users\andre\PhD\Datasets\ICDAR 2013 - Gende...,0,1,29,2214,1968,2460,...,-0.010577,0.003800,-0.024216,0.011542,0.044000,0.001486,-0.015528,-0.011971,0.027188,1
2,1,1,0,C:\Users\andre\PhD\Datasets\ICDAR 2013 - Gende...,0,1,43,1729,741,1976,...,-0.026420,-0.002679,-0.022242,-0.032118,0.033636,0.003242,-0.008457,0.004689,0.010835,2
3,1,1,1,C:\Users\andre\PhD\Datasets\ICDAR 2013 - Gende...,0,1,54,1476,1230,1722,...,-0.028946,0.003766,0.015801,-0.019674,-0.000178,-0.005016,0.005835,-0.040938,0.012388,3
4,2,0,0,C:\Users\andre\PhD\Datasets\ICDAR 2013 - Gende...,0,1,64,988,494,1235,...,-0.020056,0.002735,-0.034672,0.014498,0.035240,0.008573,-0.011511,-0.009830,0.020715,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1123,281,1,1,C:\Users\andre\PhD\Datasets\ICDAR 2013 - Gende...,0,1,16854,1488,1984,1736,...,-0.032084,0.011354,0.024554,-0.008671,0.032149,0.000281,0.027986,0.012711,0.000463,1123
1124,282,0,0,C:\Users\andre\PhD\Datasets\ICDAR 2013 - Gende...,0,1,16865,1976,494,2223,...,-0.016214,0.006361,-0.036551,0.007864,0.041912,-0.001423,-0.010395,-0.007492,0.018502,1124
1125,282,0,1,C:\Users\andre\PhD\Datasets\ICDAR 2013 - Gende...,0,1,16880,1729,1976,1976,...,-0.024396,0.003282,-0.028300,0.007869,0.058978,0.003996,-0.017661,-0.014423,0.024855,1125
1126,282,1,0,C:\Users\andre\PhD\Datasets\ICDAR 2013 - Gende...,0,1,16890,988,741,1235,...,-0.067134,-0.005375,-0.001508,0.009565,0.037255,0.011904,-0.021708,-0.004189,-0.025094,1126


In [16]:
cols_to_drop = [c for c in train_FE.columns if not(c.startswith('f') and len(c) > 1 and c[1].isdigit())]
print(cols_to_drop)

['writer', 'isEng', 'same_text', 'file_name', 'male', 'train', 'index', 'x', 'y', 'x2', 'y2', 'n_cc', 'black_ratio', 'page']


In [57]:
is_kaggle = False
with_pca=True
with_cross_validation=True
task = 'gender_detection' #, gender detection, language detection, 'writer identification'
train_on_language = 'all' #, english, all, arabic
train_on_same = 'all' #, different, all, same
n_splits=5
if task == 'language detection' and train_on_language != 'all':
    raise ValueError("For language detection, 'train_on' must be 'all'.")
if task=='gender_detection':
    target_label='male'
else:
    target_label='isEng'
if is_kaggle:
    cols_to_drop = ['writer', 'same_text', 'train','page_id','isEng','train','index','male']
else:
    cols_to_drop = [c for c in train_FE.columns if not(c.startswith('f') and len(c) > 1 and c[1].isdigit())]
    print(cols_to_drop)

'''X_train = train_FE[train_FE['train']==1].drop(columns=cols_to_drop)
y_train = train_FE[train_FE['train']==1][target_label]
X_val = train_FE[train_FE['train']==0].drop(columns=cols_to_drop)
y_val = train_FE[train_FE['train']==0][target_label]'''

['writer', 'isEng', 'same_text', 'file_name', 'male', 'train', 'index', 'x', 'y', 'x2', 'y2', 'n_cc', 'page']


"X_train = train_FE[train_FE['train']==1].drop(columns=cols_to_drop)\ny_train = train_FE[train_FE['train']==1][target_label]\nX_val = train_FE[train_FE['train']==0].drop(columns=cols_to_drop)\ny_val = train_FE[train_FE['train']==0][target_label]"

In [14]:
'''check number of 
# Count numeric columns
num_numeric_cols = train_FE.select_dtypes(include=['number']).shape[1]

# Count columns with missing values
num_missing_cols = train_FE.isnull().any().sum()

# Display results
print(f'Number of numeric columns: {num_numeric_cols}')
print(f'Number of columns with missing values: {num_missing_cols}')'''

"check number of \n# Count numeric columns\nnum_numeric_cols = train_FE.select_dtypes(include=['number']).shape[1]\n\n# Count columns with missing values\nnum_missing_cols = train_FE.isnull().any().sum()\n\n# Display results\nprint(f'Number of numeric columns: {num_numeric_cols}')\nprint(f'Number of columns with missing values: {num_missing_cols}')"

In [ ]:
'''grouped = train_FE.groupby(['isEng', 'same_text'])
# Display the first five elements for each group in 'grouped'
for group_name, group_df in grouped:
    print(f"Group: {group_name}")
    display(group_df.head(5))
# groups 0,0 0,1 1,0 1,1
# Select two group names (replace with actual group keys as needed)
group_keys = list(grouped.groups.keys()) 

# Get the DataFrames for the first two groups
df_arab_diff = grouped.get_group(group_keys[0])
df_arab_same = grouped.get_group(group_keys[1])
df_eng_diff = grouped.get_group(group_keys[2])
df_eng_same = grouped.get_group(group_keys[3])


# Concatenate the two DataFrames
concatenated_df = pd.concat([df1, df2], ignore_index=True)

print(concatenated_df.shape)
concatenated_df.head()''' 

Group: (np.int64(0), np.int64(0))


,writer,isEng,same_text,file_name,male,train,index,x,y,x2,...,f759,f760,f761,f762,f763,f764,f765,f766,f767,f768
0,1,0,0,unzipped/1_50/0001_1.jpg,0,1,0,0,493,493,...,-0.275631,0.058159,-0.001750,-0.269634,0.219695,0.180648,-0.243915,-0.396190,0.520203,-1.462304
1,1,0,0,unzipped/1_50/0001_1.jpg,0,1,1,493,493,986,...,-0.271600,0.066998,0.031707,-0.261003,0.210312,0.202038,-0.280518,-0.420835,0.522806,-1.437481
2,1,0,0,unzipped/1_50/0001_1.jpg,0,1,2,1479,493,1972,...,-0.221603,0.073357,-0.017823,-0.191817,0.199360,0.190194,-0.270286,-0.385847,0.498827,-1.451828
3,1,0,0,unzipped/1_50/0001_1.jpg,0,1,3,986,493,1479,...,-0.225266,0.062873,0.013257,-0.242814,0.213373,0.197258,-0.285037,-0.393540,0.503814,-1.470287
4,1,0,0,unzipped/1_50/0001_1.jpg,0,1,4,1972,493,2465,...,-0.213585,0.063182,-0.015335,-0.204292,0.231909,0.235870,-0.279547,-0.388177,0.468346,-1.508323


Group: (np.int64(0), np.int64(1))


,writer,isEng,same_text,file_name,male,train,index,x,y,x2,...,f759,f760,f761,f762,f763,f764,f765,f766,f767,f768
5,1,0,1,unzipped/1_50/0001_2.jpg,0,1,5,493,1479,986,...,-0.243933,0.081438,-0.026993,-0.202966,0.186488,0.202112,-0.260245,-0.378924,0.496770,-1.494687
6,1,0,1,unzipped/1_50/0001_2.jpg,0,1,6,986,986,1479,...,-0.288985,0.087640,0.049662,-0.208533,0.235510,0.152196,-0.247914,-0.395293,0.474001,-1.424915
7,1,0,1,unzipped/1_50/0001_2.jpg,0,1,7,1479,1479,1972,...,-0.270145,0.070758,-0.028458,-0.210834,0.227700,0.216413,-0.248142,-0.398217,0.536445,-1.459662
8,1,0,1,unzipped/1_50/0001_2.jpg,0,1,8,493,986,986,...,-0.287041,0.121514,-0.008135,-0.220779,0.179901,0.202289,-0.235622,-0.422261,0.486641,-1.461829
9,1,0,1,unzipped/1_50/0001_2.jpg,0,1,9,1479,986,1972,...,-0.254537,0.080850,-0.013784,-0.200407,0.215601,0.180609,-0.256348,-0.401637,0.495235,-1.471664


Group: (np.int64(1), np.int64(0))


,writer,isEng,same_text,file_name,male,train,index,x,y,x2,...,f759,f760,f761,f762,f763,f764,f765,f766,f767,f768
10,1,1,0,unzipped/1_50/0001_3.jpg,0,1,10,1976,494,2470,...,-0.254266,0.127310,-0.011507,-0.097587,0.210800,0.329141,-0.268400,-0.457523,0.487944,-1.518100
11,1,1,0,unzipped/1_50/0001_3.jpg,0,1,11,494,494,988,...,-0.310239,0.088337,0.010448,-0.130279,0.240916,0.205589,-0.290277,-0.437706,0.481369,-1.452869
12,1,1,0,unzipped/1_50/0001_3.jpg,0,1,12,1482,494,1976,...,-0.274421,0.140499,0.017490,-0.153239,0.227730,0.235040,-0.259118,-0.435480,0.512737,-1.424407
13,1,1,0,unzipped/1_50/0001_3.jpg,0,1,13,988,988,1482,...,-0.239550,0.093602,-0.017014,-0.179545,0.203855,0.343389,-0.240150,-0.419020,0.517605,-1.482068
14,1,1,0,unzipped/1_50/0001_3.jpg,0,1,14,988,494,1482,...,-0.234004,0.037380,-0.018710,-0.169868,0.214705,0.196334,-0.324915,-0.407791,0.469555,-1.493571


Group: (np.int64(1), np.int64(1))


,writer,isEng,same_text,file_name,male,train,index,x,y,x2,...,f759,f760,f761,f762,f763,f764,f765,f766,f767,f768
15,1,1,1,unzipped/1_50/0001_4.jpg,0,1,15,1479,1972,1972,...,-0.230391,0.100809,0.022167,-0.217382,0.205409,0.214047,-0.273079,-0.402834,0.488653,-1.480792
16,1,1,1,unzipped/1_50/0001_4.jpg,0,1,16,493,1479,986,...,-0.281213,0.085982,0.038979,-0.181102,0.202569,0.286283,-0.251128,-0.440668,0.483774,-1.501802
17,1,1,1,unzipped/1_50/0001_4.jpg,0,1,17,986,2465,1479,...,-0.275889,0.120276,0.021062,-0.174980,0.236136,0.237443,-0.236544,-0.449596,0.498428,-1.434339
18,1,1,1,unzipped/1_50/0001_4.jpg,0,1,18,986,1972,1479,...,-0.212950,0.062496,0.024899,-0.171433,0.244673,0.180122,-0.272156,-0.365500,0.487081,-1.443907
19,1,1,1,unzipped/1_50/0001_4.jpg,0,1,19,493,1972,986,...,-0.276774,0.106780,0.034823,-0.214603,0.227066,0.222292,-0.244637,-0.424428,0.481181,-1.470038


In [58]:
train_FE=select_groups(train_FE,select_column='train', 
                       train_on_language=train_on_language, train_on_same=train_on_same)

train_FE_selected = train_FE[train_FE['train']==1].copy()
X = train_FE[train_FE['train']==1].drop(columns=cols_to_drop)
y = train_FE[train_FE['train']==1][target_label]

writers = train_FE[train_FE['train']==1]['writer']-1
pages = train_FE[train_FE['train']==1]['page']

gkf = GroupKFold(n_splits=n_splits)

In [14]:
print(len(X), len(y), len(writers))
print("Min writer ID:", writers.min())
print("Max writer ID:", writers.max())

5640 5640 5640
Min writer ID: 0
Max writer ID: 281


In [84]:
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

selected_model = 'logreg' # 'gbm', 'lgbm', 'xgb', 'rf', 'mlp', 'dt', 'logreg'
# Define the models
gbm_classifier = GradientBoostingClassifier(
    n_estimators=100, #100 is standard 
    learning_rate=0.1,  
    max_depth=3,  
    random_state=42
)

#cat = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0, random_state=42)

lgbm = lgb.LGBMClassifier(n_estimators=10, learning_rate=0.1, max_depth=-1, random_state=42)
'''lgbm = lgb.LGBMClassifier(
    n_estimators=100,
    learning_rate=0.05,
    max_depth=6,
    num_leaves=20,
    min_child_samples=30,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=1.0,
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1
)'''

xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

rf = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)

mlp = MLPClassifier(hidden_layer_sizes=(128), activation='relu', solver='adam',
                    max_iter=200, random_state=42, early_stopping=True, validation_fraction=0.1, n_iter_no_change=10)

dt = DecisionTreeClassifier(max_depth=3, min_samples_split=5, min_samples_leaf=2, ccp_alpha=0.01, random_state=42)

logreg = LogisticRegression(max_iter=1000, random_state=42)

feature_extraction_model = {'gbm': gbm_classifier, 'lgbm': lgbm, 'xgb': xgb, 'rf':rf, 'mlp':mlp, 'dt':dt,'logreg':logreg}

In [85]:
import time

#feature_extraction_model = {'gbm': gbm_classifier, 'cat': cat, 'lgbm': lgbm, 'xgb': xgb}
# Define the pipeline
if with_pca:
    from sklearn.decomposition import PCA
    pca = PCA(n_components=0.95)  #384
    pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Normalize features
        ('pca', pca),  # Apply PCA
        (selected_model, feature_extraction_model[selected_model])  # Train GBM classifier
    ])
else:    
    pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Normalize features
        (selected_model, feature_extraction_model[selected_model])  # Train GBM classifier
    ])

In [107]:
from collections import Counter
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import shuffle

def ensembled_predictions(base_preds,writers,mode='majority_vote',probs=None):
    pred_df = pd.DataFrame({
            'writer': writers,
            'pred': base_preds
        })
    if probs is not None:
        probs = np.abs(probs - 0.5) / 0.5
        pred_df['prob'] = probs
    if mode == 'majority_vote':
        writer_preds = pred_df.groupby('writer')['pred'].agg(
            lambda x: Counter(x).most_common(1)[0][0]
        )
    elif mode == 'weighted_vote':
        if probs is None:
            raise ValueError("For 'weighted_vote', 'probs' must be provided.")
        writer_preds = pred_df.groupby('writer', group_keys=False).apply(
            lambda x: pd.Series({
                'writer_pred': round((x['pred'] * x['prob']).sum() / x['prob'].sum())
            })
        )['writer_pred'].astype(int)
    elif mode == 'most_probable':
        if probs is None:
            raise ValueError("For 'most_probable', 'probs' must be provided.")
        writer_preds = pred_df.groupby('writer', group_keys=False).apply(
            lambda x: x.loc[x['prob'].idxmax(), 'pred']
        )

    # Step 5: Map writer-level prediction back to each sample
    #final_preds = writers.map(writer_preds)
    return writer_preds#final_preds.values

def group_labels(y, writers):
    """
    Groups labels by writer.
    """
    grouped_labels = y.groupby(writers).agg(lambda x: Counter(x).most_common(1)[0][0])
    return grouped_labels

def compute_accuracies(y_true, y_pred, y_prob, pages, writers):
    """
    Computes accuracy for each writer.
    """
    accuracies = {}
    accuracies['individual'] = accuracy_score(y_true, y_pred)

    grouped_true = group_labels(y_true, pages)
    grouped_pred = ensembled_predictions(y_pred, pages)
    accuracies['ensembled'] = accuracy_score(grouped_true, grouped_pred)

    grouped_pred = ensembled_predictions(y_pred, pages, mode='weighted_vote',probs=y_prob)
    accuracies['ensembled_weighted'] = accuracy_score(grouped_true, grouped_pred)

    grouped_pred = ensembled_predictions(y_pred, pages, mode='most_probable',probs=y_prob)
    accuracies['ensembled_most_probable'] = accuracy_score(grouped_true, grouped_pred)

    grouped_true_writers = group_labels(y_true, writers)
    grouped_pred_writers = ensembled_predictions(y_pred, writers)
    accuracies['ensembled_writers'] = accuracy_score(grouped_true_writers, grouped_pred_writers)
    
    return accuracies

In [176]:
#test ensemble predictions function
base_preds=pd.Series([1,0,0,0,1])
test_writers= pd.Series([1, 1, 2, 2, 1])
print(ensembled_predictions(base_preds, test_writers))
base_labels = pd.Series([1,1,0,0,1])
print(group_labels(base_labels, test_writers))

writer
1    1
2    0
Name: pred, dtype: int64
1    1
2    0
dtype: int64


In [212]:
#test grouping
test_indices = np.random.choice(int(len(writers)*4/5), size=10, replace=False)
for train_idx, val_idx in gkf.split(X, y, groups=writers):
    X_train, X_val = X.loc[train_idx], X.loc[val_idx]
    y_train, y_val = y.loc[train_idx], y.loc[val_idx]
    print(val_idx[:30])
    print(list(writers[val_idx][:30]))
    print("Number of unique writers in val_idx:", writers[val_idx].nunique(), "->", writers[val_idx].nunique()/ len(writers.unique()))
    print("---------------------------------")
    for test_i in test_indices:
        index=train_idx[test_i]
        print(index)
        print(writers[index])
        label= y[index]
        print(label)
        X_test= X.loc[index]
        print('----------')
    break

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19 100 101 102 103 104 105 106 107 108 109]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
Number of unique writers in val_idx: 57 -> 0.20212765957446807
---------------------------------
5018
250
0
----------
2648
132
1
----------
3427
171
0
----------
1728
86
0
----------
3171
158
1
----------
893
44
0
----------
970
48
1
----------
3572
178
1
----------
2304
115
0
----------
643
32
0
----------


In [108]:
# Measure the start time
start_time = time.time()
print(f"Starting model cross-val...")
cross_val_accuracies = {"IF": [], "OOF": []}
for train_idx, val_idx in gkf.split(X, y, groups=writers):
    #print(train_idx)
    X_train, X_val = X.loc[train_idx], X.loc[val_idx]
    y_train, y_val = y.loc[train_idx], y.loc[val_idx]
    X_train, y_train, writers_train, pages_train = shuffle(
        X_train, y_train, writers.loc[train_idx], pages.loc[train_idx], random_state=42
    )
    print(f"Training on {len(X_train)} samples, validating on {len(X_val)} writers {len(writers_train)}")

    # Fit the model on training data
    pipeline.fit(X_train.values, y_train)
    y_prob= pipeline.predict_proba(X_train.values)[:,1]
    #y_pred = pipeline.predict(X_train.values)
    y_pred=(y_prob>= 0.5).astype(int)
    accuracies = compute_accuracies(y_train, y_pred, y_prob,pages_train,writers_train)
    cross_val_accuracies["IF"].append(accuracies)
    print('IF accuracies', accuracies)
    #print(f"IF accuracies -> Ensembled accuracy: {accuracies['ensembled']:.3f}, Individual accuracy: {accuracies['individual']:.3f}")

    y_prob= pipeline.predict_proba(X_val.values)[:,1]
    #y_pred = pipeline.predict(X_val.values)
    y_pred=(y_prob >= 0.5).astype(int)
    accuracies = compute_accuracies(y_val, y_pred, y_prob,pages.loc[val_idx], writers.loc[val_idx])
    cross_val_accuracies["OOF"].append(accuracies)
    print('OOF accuracies', accuracies)
    #print(f"OOF accuracies -> Ensembled accuracy writers: {accuracies['ensembled_writers']:.3f}, Ensembled accuracy: {accuracies['ensembled']:.3f}, Individual accuracy: {accuracies['individual']:.3f}")
    #print(f"Training on {len(X_train)} samples, validating on {len(X_val)} samples")
    subgroup_accuracies=compute_subgroup_accuracies(pipeline, train_FE_selected.loc[val_idx])
    print(f"Subgroup accuracies: {subgroup_accuracies}")
    print("-----------------------------*****************************---------------------------------------------")

# Measure the end time
end_time = time.time()

# Calculate the time taken
time_taken_cross_val = end_time - start_time
print(f"Time taken to fit the model: {time_taken_cross_val:.2f} seconds")

Starting model cross-val...
Training on 4500 samples, validating on 1140 writers 4500


C:\Users\andre\AppData\Local\Temp\ipykernel_26080\3402381078.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  writer_preds = pred_df.groupby('writer', group_keys=False).apply(
C:\Users\andre\AppData\Local\Temp\ipykernel_26080\3402381078.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  writer_preds = pred_df.groupby('writer', group_keys=False).apply(
C:\Users\andre\AppData\Local\Temp\ipykernel_26080\

IF accuracies {'individual': 0.7686666666666667, 'ensembled': 0.8088888888888889, 'ensembled_weighted': 0.82, 'ensembled_most_probable': 0.82, 'ensembled_writers': 0.8844444444444445}
OOF accuracies {'individual': 0.7087719298245614, 'ensembled': 0.7149122807017544, 'ensembled_weighted': 0.7192982456140351, 'ensembled_most_probable': 0.7192982456140351, 'ensembled_writers': 0.8421052631578947}
Evaluating group: ('english', 'different')
Accuracy on subgroup -> Ensembled accuracy writer: 0.719, Ensembled accuracy: 0.719, Individual accuracy: 0.684
----------------------------------------
Evaluating group: ('english', 'same')


C:\Users\andre\AppData\Local\Temp\ipykernel_26080\3402381078.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  writer_preds = pred_df.groupby('writer', group_keys=False).apply(
C:\Users\andre\AppData\Local\Temp\ipykernel_26080\3402381078.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  writer_preds = pred_df.groupby('writer', group_keys=False).apply(
C:\Users\andre\AppData\Local\Temp\ipykernel_26080\

Accuracy on subgroup -> Ensembled accuracy writer: 0.719, Ensembled accuracy: 0.719, Individual accuracy: 0.733
----------------------------------------
Evaluating group: ('arabic', 'different')
Accuracy on subgroup -> Ensembled accuracy writer: 0.719, Ensembled accuracy: 0.719, Individual accuracy: 0.719
----------------------------------------
Evaluating group: ('arabic', 'same')
Accuracy on subgroup -> Ensembled accuracy writer: 0.702, Ensembled accuracy: 0.702, Individual accuracy: 0.698
----------------------------------------
Evaluating group: ('english', 'all')
Accuracy on subgroup -> Ensembled accuracy writer: 0.719, Ensembled accuracy: 0.719, Individual accuracy: 0.709
----------------------------------------
Evaluating group: ('arabic', 'all')


...

C:\Users\andre\AppData\Local\Temp\ipykernel_26080\3402381078.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  writer_preds = pred_df.groupby('writer', group_keys=False).apply(
C:\Users\andre\AppData\Local\Temp\ipykernel_26080\3402381078.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  writer_preds = pred_df.groupby('writer', group_keys=False).apply(
C:\Users\andre\AppData\Local\Temp\ipykernel_26080\

Accuracy on subgroup -> Ensembled accuracy writer: 0.696, Ensembled accuracy: 0.696, Individual accuracy: 0.664
----------------------------------------
Evaluating group: ('arabic', 'different')
Accuracy on subgroup -> Ensembled accuracy writer: 0.554, Ensembled accuracy: 0.554, Individual accuracy: 0.557
----------------------------------------
Evaluating group: ('arabic', 'same')
Accuracy on subgroup -> Ensembled accuracy writer: 0.786, Ensembled accuracy: 0.786, Individual accuracy: 0.696
----------------------------------------
Evaluating group: ('english', 'all')
Accuracy on subgroup -> Ensembled accuracy writer: 0.732, Ensembled accuracy: 0.688, Individual accuracy: 0.652
----------------------------------------
Evaluating group: ('arabic', 'all')


C:\Users\andre\AppData\Local\Temp\ipykernel_26080\3402381078.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  writer_preds = pred_df.groupby('writer', group_keys=False).apply(
C:\Users\andre\AppData\Local\Temp\ipykernel_26080\3402381078.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  writer_preds = pred_df.groupby('writer', group_keys=False).apply(
C:\Users\andre\AppData\Local\Temp\ipykernel_26080\

Accuracy on subgroup -> Ensembled accuracy writer: 0.696, Ensembled accuracy: 0.670, Individual accuracy: 0.627
----------------------------------------
Evaluating group: ('all', 'different')
Accuracy on subgroup -> Ensembled accuracy writer: 0.625, Ensembled accuracy: 0.616, Individual accuracy: 0.598
----------------------------------------
Evaluating group: ('all', 'same')
Accuracy on subgroup -> Ensembled accuracy writer: 0.786, Ensembled accuracy: 0.741, Individual accuracy: 0.680
----------------------------------------
Subgroup accuracies: {'english,different': {'individual': 0.6392857142857142, 'ensembled': 0.6785714285714286, 'ensembled_weighted': 0.6964285714285714, 'ensembled_most_probable': 0.7142857142857143, 'ensembled_writers': 0.6785714285714286}, 'english,same': {'individual': 0.6642857142857143, 'ensembled': 0.6964285714285714, 'ensembled_weighted': 0.6964285714285714, 'ensembled_most_probable': 0.7321428571428571, 'ensembled_writers': 0.6964285714285714}, 'arabic,dif

C:\Users\andre\AppData\Local\Temp\ipykernel_26080\3402381078.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  writer_preds = pred_df.groupby('writer', group_keys=False).apply(
C:\Users\andre\AppData\Local\Temp\ipykernel_26080\3402381078.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  writer_preds = pred_df.groupby('writer', group_keys=False).apply(


# retraining

In [74]:
start_time = time.time()
print(f"Retraining on all training data...")
pipeline.fit(X, y)
end_time = time.time()

# Calculate the time taken
time_taken = end_time - start_time
print(f"Time taken to fit the model: {time_taken:.2f} seconds")

Retraining on all training data...
[LightGBM] [Info] Number of positive: 2780, number of negative: 2860
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004031 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29325
[LightGBM] [Info] Number of data points in the train set: 5640, number of used features: 115
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.492908 -> initscore=-0.028371
[LightGBM] [Info] Start training from score -0.028371
Time taken to fit the model: 1.28 seconds


In [75]:
if with_pca:
    pca = pipeline.named_steps['pca']
    print(f"Explained variance ratio by PCA: {pca.explained_variance_ratio_}")
    print(f"Number of features used after PCA: {pca.n_components_}")

Explained variance ratio by PCA: [0.27086154 0.14895208 0.08148499 0.05477929 0.03899795 0.0340924
 0.02889819 0.02050937 0.018703   0.01674469 0.01499274 0.01372273
 0.01118698 0.01065082 0.0102069  0.00928136 0.00741702 0.00688391
 0.00668457 0.00538263 0.00525704 0.00501755 0.00490214 0.00447999
 0.00414312 0.00391832 0.00382237 0.00373094 0.00353072 0.00348215
 0.00338362 0.0032251  0.00306907 0.00282812 0.00262151 0.00247765
 0.00233217 0.00233077 0.00228889 0.00214704 0.00209126 0.00200259
 0.00192751 0.0018739  0.00185081 0.00175507 0.00171918 0.0016803
 0.00166548 0.00159953 0.00153799 0.00150362 0.00146323 0.00145204
 0.00136959 0.00133634 0.00129349 0.00126573 0.0012568  0.00122379
 0.00117126 0.00113983 0.00111068 0.00106524 0.0010635  0.00104219
 0.001025   0.00099638 0.00098295 0.00094427 0.00092341 0.00090255
 0.00089064 0.00086622 0.0008609  0.00084022 0.00080507 0.00079492
 0.00078686 0.00077203 0.00076686 0.00076166 0.00073198 0.0007137
 0.0007078  0.00068814 0.0006832

In [76]:
from itertools import combinations
subgroup_accuracies = {}
# Predict on train data
y_pred = pipeline.predict(X)
accuracies = compute_accuracies(y, y_pred, writers)
subgroup_accuracies['english+arabic,different+same'] = accuracies
print(f"Training Accuracy on all training data -> Ensembled accuracy: {accuracies['ensembled']:.3f}, Individual accuracy: {accuracies['individual']:.3f}")
training_accuracy = accuracy_score(y, y_pred)
print('----------------------------------------')
groups = [('english','different'), ('english','same'), ('arabic','different'), ('arabic','same')]
group_sizes = []
for group in groups:
    print(f"Evaluating group: {group}")
    train_FE=select_groups(train_FE,select_column='train', 
                        train_on_language=group[0], train_on_same=group[1])
    X_s = train_FE[train_FE['train']==1].drop(columns=cols_to_drop)
    y_s = train_FE[train_FE['train']==1][target_label]
    writers_s = train_FE[train_FE['train']==1]['writer']-1
    group_sizes.append(len(X_s))
    y_pred = pipeline.predict(X_s)
    accuracies = compute_accuracies(y_s, y_pred, writers_s)
    subgroup_accuracies[f'{group[0]},{group[1]}'] = accuracies
    print(f"Accuracy on subgroup -> Ensembled accuracy: {accuracies['ensembled']:.3f}, Individual accuracy: {accuracies['individual']:.3f}")
    print('----------------------------------------')
groups_joined = [(0,1),(2,3),(0,2),(1,3)]
for group in groups_joined:
    language_1,same_1 = groups[group[0]]
    language_2,same_2 = groups[group[1]]
    if language_1 == language_2:
        first= f'{language_1}'
    else:
        first = f'{language_1}+{language_2}'
    if same_1 == same_2:
        second = f'{same_1}'
    else:
        second = f'{same_1}+{same_2}'
    group_name= f'{first},{second}'
    print(f"Evaluating group: {group_name}")
    sub_group_sizes = [group_sizes[group[0]], group_sizes[group[1]]]
    sub_group_accuracies = [subgroup_accuracies[f'{language_1},{same_1}'], subgroup_accuracies[f'{language_2},{same_2}']]
    accuracies['ensembled'] = np.average([g['ensembled'] for g in sub_group_accuracies], weights=sub_group_sizes)
    accuracies['individual'] = np.average([g['individual'] for g in sub_group_accuracies], weights=sub_group_sizes)
    subgroup_accuracies[f'{group_name}'] = accuracies
    print(f"Accuracy on subgroup -> Ensembled accuracy: {accuracies['ensembled']:.3f}, Individual accuracy: {accuracies['individual']:.3f}")
    print('----------------------------------------')
'''# Iterate over all possible non-empty subgroups of group indices [0, 1, 2, 3]
all_indices = [0, 1, 2, 3]
for r in [2,4]:
    for idxs in combinations(all_indices, r):
        subgroup_sizes = [group_sizes[i] for i in idxs]
        subgroup_accuracies = [accuracies[i] for i in idxs]
        weighted_mean = np.average(subgroup_accuracies, weights=subgroup_sizes)
        languages = [groups[i][0] for i in idxs]
        same_texts = [groups[i][1] for i in idxs]
        print(f"Weighted mean accuracy for {languages} and {same_texts}: {weighted_mean:.4f}")'''


c:\Users\andre\anaconda3\envs\GeneralPurposeML\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\andre\anaconda3\envs\GeneralPurposeML\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\andre\anaconda3\envs\GeneralPurposeML\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\andre\anaconda3\envs\GeneralPurposeML\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Training Accuracy on all training data -> Ensembled accuracy: 0.926, Individual accuracy: 0.769
----------------------------------------
Evaluating group: ('english', 'different')
Accuracy on subgroup -> Ensembled accuracy: 0.876, Individual accuracy: 0.786
----------------------------------------
Evaluating group: ('english', 'same')
Accuracy on subgroup -> Ensembled accuracy: 0.833, Individual accuracy: 0.772
----------------------------------------
Evaluating group: ('arabic', 'different')
Accuracy on subgroup -> Ensembled accuracy: 0.840, Individual accuracy: 0.754
----------------------------------------
Evaluating group: ('arabic', 'same')
Accuracy on subgroup -> Ensembled accuracy: 0.844, Individual accuracy: 0.764
----------------------------------------
Evaluating group: english,different+same
Accuracy on subgroup -> Ensembled accuracy: 0.855, Individual accuracy: 0.779
----------------------------------------
Evaluating group: arabic,different+same
Accuracy on subgroup -> Ens

c:\Users\andre\anaconda3\envs\GeneralPurposeML\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


'# Iterate over all possible non-empty subgroups of group indices [0, 1, 2, 3]\nall_indices = [0, 1, 2, 3]\nfor r in [2,4]:\n    for idxs in combinations(all_indices, r):\n        subgroup_sizes = [group_sizes[i] for i in idxs]\n        subgroup_accuracies = [accuracies[i] for i in idxs]\n        weighted_mean = np.average(subgroup_accuracies, weights=subgroup_sizes)\n        languages = [groups[i][0] for i in idxs]\n        same_texts = [groups[i][1] for i in idxs]\n        print(f"Weighted mean accuracy for {languages} and {same_texts}: {weighted_mean:.4f}")'

In [224]:
print(subgroup_accuracies)

{'english+arabic,different+same': {'ensembled': 0.8368794326241135, 'individual': 0.7716312056737589}, 'english,different': {'ensembled': 0.8794326241134752, 'individual': 0.7936170212765957}, 'english,same': {'ensembled': 0.8333333333333334, 'individual': 0.7780141843971631}, 'arabic,different': {'ensembled': 0.8404255319148937, 'individual': 0.7936170212765957}, 'arabic,same': {'ensembled': np.float64(0.8466312056737588), 'individual': np.float64(0.7858156028368795)}, 'english,different+same': {'ensembled': np.float64(0.8466312056737588), 'individual': np.float64(0.7858156028368795)}, 'arabic,different+same': {'ensembled': np.float64(0.8466312056737588), 'individual': np.float64(0.7858156028368795)}, 'english+arabic,different': {'ensembled': np.float64(0.8466312056737588), 'individual': np.float64(0.7858156028368795)}, 'english+arabic,same': {'ensembled': np.float64(0.8466312056737588), 'individual': np.float64(0.7858156028368795)}}


In [77]:
experiment = datetime.now().strftime("%Y%m%d_%H%M%S")
out_dir = os.path.join(source_path, "outputs", "logs")
# Example usage:
LOG_OUT_FILE = out_dir+"\\feature_extraction_metadata_log.json"
print(f"Log file path: {LOG_OUT_FILE}")
file_IO.add_or_update_experiment(
    experiment, LOG_OUT_FILE,
    custom_metadata={
        "type of preprocessing": extracted_from,
        "original raw file": source_data,
        "input file": input_file_name,
        "FE model": model_used,
        "FE transform": transform_used,
        "classifier model": selected_model,
        "model_params": feature_extraction_model[selected_model].get_params(),
        "n_splits": n_splits,
        "train_on_language": train_on_language,
        "train_on_same": train_on_same,
        "task": task,
        "with cross validation": with_cross_validation,
        "with PCA": with_pca,
        "training time for cross-validation": time_taken_cross_val,
        "training time for final model": time_taken,
        "cross_val_accuracies": cross_val_accuracies,
        "subgroup_accuracies": subgroup_accuracies,
        "is_kaggle": is_kaggle,
        "test": 'this is a test column',
        "description": ''' I am training a classifier on the feature vectors extracted by a deep model
        I am evaluating the results on subsets of the training data, based on language and same/different text.''' 
    }
)

Log file path: c:\Users\andre\VsCode\PD related projects\gender_detection\outputs\logs\feature_extraction_metadata_log.json
Updated log for experiment 20250528_144225


# easy access

In [5]:
def reload_modules():
    import importlib
    import utils.file_IO as file_IO
    

    importlib.reload(file_IO)

    return file_IO

file_IO = reload_modules()

In [6]:
def select_groups(train_FE,select_column='train', train_on_language='arabic', train_on_same='same'):
    train_FE[select_column] = 1
    if train_on_language=='arabic':
        train_FE.loc[train_FE['isEng'] == 1, select_column] = 0 #remove english
    elif train_on_language=='english':
        train_FE.loc[train_FE['isEng'] == 0, select_column] = 0 #remove arabic
    else:
        pass
    if train_on_same=='same':
        train_FE.loc[train_FE['same_text'] == 0, select_column] = 0 #remove different texts
    elif train_on_same=='different':
        train_FE.loc[train_FE['same_text'] == 1, select_column] = 0 #remove same texts
    else:
        pass
    return train_FE


In [7]:
def compute_subgroup_accuracies(pipeline, train_df):
    subgroup_accuracies = {}
    groups = [('english','different'), ('english','same'), ('arabic','different'), ('arabic','same'),
              ('english','all'), ('arabic','all'), ('all','different'), ('all','same')]
    group_sizes = []
    acc_keys=None
    for group in groups:
        print(f"Evaluating group: {group}")
        train_df=select_groups(train_df,select_column='train', 
                            train_on_language=group[0], train_on_same=group[1])
        X_s = train_df[train_df['train']==1].drop(columns=cols_to_drop)
        y_s = train_df[train_df['train']==1][target_label]
        writers_s = train_df[train_df['train']==1]['writer']-1
        pages_s = train_df[train_df['train']==1]['page']
        group_sizes.append(len(X_s))

        y_prob= pipeline.predict_proba(X_s.values)[:,1]
        #y_pred = pipeline.predict(X_s.values)
        y_pred=(y_prob >= 0.5).astype(int)
        accuracies = compute_accuracies(y_s, y_pred, y_prob, pages_s,writers_s)
        subgroup_accuracies[f'{group[0]},{group[1]}'] = accuracies
        if acc_keys==None:
            acc_keys = list(accuracies.keys())
        print(f"Accuracy on subgroup -> Ensembled accuracy writer: {accuracies['ensembled_writers']:.3f}, Ensembled accuracy: {accuracies['ensembled']:.3f}, Individual accuracy: {accuracies['individual']:.3f}")
        print('----------------------------------------')
    return subgroup_accuracies
    '''groups_joined = [(0,1),(2,3),(0,2),(1,3)]
    for group in groups_joined:
        language_1,same_1 = groups[group[0]]
        language_2,same_2 = groups[group[1]]
        if language_1 == language_2:
            first= f'{language_1}'
        else:
            first = f'{language_1}+{language_2}'
        if same_1 == same_2:
            second = f'{same_1}'
        else:
            second = f'{same_1}+{same_2}'
        group_name= f'{first},{second}'
        print(f"Evaluating group: {group_name}")
        sub_group_sizes = [group_sizes[group[0]], group_sizes[group[1]]]
        sub_group_accuracies = [subgroup_accuracies[f'{language_1},{same_1}'], subgroup_accuracies[f'{language_2},{same_2}']]
        for key in acc_keys:
            accuracies[key] = np.average([g[key] for g in sub_group_accuracies], weights=sub_group_sizes)
        subgroup_accuracies[f'{group_name}'] = accuracies
        print(f"Accuracy on subgroup -> Ensembled accuracy writer: {accuracies['ensembled_writers']:.3f}, Ensembled accuracy: {accuracies['ensembled']:.3f}, Individual accuracy: {accuracies['individual']:.3f}")
        print('----------------------------------------')'''